# Analiza sieci tramwajowej w Poznaniu

Przedmiotem analizy jest sieć tramwajowa w Poznaniu. Dane o przejazdach pozyskaliśmy samodzielnie ze strony ZTM: https://www.ztm.poznan.pl/pl/rozklad-jazdy/ przy użyciu skryptu skrapującego. Dane o lokalizacji przystanków pozyskaliśmy ręcznie z Google Maps, ponieważ nigdzie niestety takie dane nie były dostępne.

### Pytania o charakterze biznesowym:
* Które części miasta mogą cierpieć z powodu nierozbudowanej sieci tramwajowej?
* Które węzły są w sieci najważniejsze z punktu widzenia ewentualnego wyłączenia z użycia?

## 0. Importy

In [1]:
import networkx
import pprint
from typing import Mapping, List, Tuple

## 1. Wczytanie danych

In [2]:
def read_routes() -> Mapping[str, List]:
    routes = {}
    
    trams = list(range(1, 19)) + [201]
    for tram in trams:
        stops = None
        with open(f'./data/{tram}there') as f:
            stops = f.read().splitlines()
            stops = [s.strip() for s in stops]
        routes[f"{tram}there"] = stops

        with open(f'./data/{tram}back') as f:
            stops = f.read().splitlines()
            stops = [s.strip() for s in stops]
        routes[f"{tram}back"] = stops
    return routes

def read_coords() -> Mapping[str, Tuple]:
    result = {}
    with open(f'./data/coords') as f:
        lines = f.read().splitlines()
        for line in lines:
            line = line.strip()
            stop_name, x, y  = line.split(",")
            result[stop_name] = (x,y)
    return result

In [3]:
route_to_stops : Mapping[str, List] = read_routes()

In [4]:
stop_to_location : Mapping[str, Tuple] = read_coords()

In [5]:
# just to check if we have all coords
for r, stops in route_to_stops.items():
    for stop in stops:
        if stop not in stop_to_location:
            print(stop)

## 2. Utworzenie skierowanego Multigrafu

In [6]:
G = networkx.MultiDiGraph()

for stop_name, coords in stop_to_location.items():
    G.add_node(stop_name, location=coords)

# Adding edges
for k, stops_list in route_to_stops.items():
    edges = [(stops_list[i],stops_list[i+1]) for i in range(len(stops_list)-1)]
    G.add_edges_from(edges)

In [7]:
list(G)

['Junikowo (JUNI42)',
 'Franowo (FRWO42)',
 'Szwajcarska (SZWA42)',
 'Szwedzka (SZWE42)',
 'Piaśnicka/Kurlandzka (PK42)',
 'Piaśnicka Rynek (PIAR42)',
 'Os. Czecha (OCZ71)',
 'Żegrze III (ZIII41)',
 'Żegrze II (ZII41)',
 'Żegrze I (ZI41)',
 'Rondo Żegrze (RZEG43)',
 'Rondo Żegrze (RZEG41)',
 'Rondo Starołęka (RSTA47)',
 'Rondo Starołęka (RSTA43)',
 'Hetmańska Wiadukt (HEWI41)',
 'Rolna (ROLN41)',
 'Traugutta (TRAU43)',
 'Kolejowa (KOLE41)',
 'Głogowska/Hetmańska (GLHE43)',
 'Arciszewskiego (ARCI41)',
 'Arena (AREN41)',
 'Rondo Nowaka-Jeziorańskiego (RONJ41)',
 'Ostroroga (OSTR41)',
 'Grochowska (GROC41)',
 'Drzewieckiego (DRZE41)',
 'Stadion Miejski - Punkt Szczepień (STMI41)',
 'Węgorka (WEGO42)',
 'Rondo Skubiszewskiego (ROSK41)',
 'Budziszyńska (BUDZ41)',
 'Grotkowska (GROT71)',
 'Cmentarna (CMNA41)',
 'Junikowo (JUNI43)',
 'Junikowo (JUNI41)',
 'Ogrody (OGDY42)',
 'Żeromskiego (ZERO72)',
 'Polna (PONA72)',
 'Rynek Jeżycki (RJEZ72)',
 'Kraszewskiego (KRAS42)',
 'Stare Zoo (SZOO72)',

### Rozkład stopni wyjściowych i wejściowych

### Rozkład długości najkrótszych ścieżek

### Rozkład pośrednictwa

### Rozkład lokalnych współczynników grupowania

### Miary oceny sieci: gęstość, współczynnik centralizacji, promień 

### Podział sieci na moduły

### Trzy wizualizacje